## **通过时间反向传播**

在上一节中，我们知道，如果不裁剪梯度，模型将无法正常训练。为了深刻理解这一现象，本节将介绍循环神经网络中梯度的计算和存储方法，即通过时间反向传播（back-propagation through time)。

正向传播和反向传播相互依赖。

正向传播在循环神经网络中比较直观，而通过时间反向传播其实是反向传播在循环神经网络中的具体应用。

我们需要将循环网络按时间步展开，从而得到模型变量和参数之间的依赖关系，并依据链式法则应用反向传播计算并存储梯度。


### <font color='blue'>**1、定义模型**</font>

考虑一个无偏差项的循环神经网络，且激活函数为恒等映射 $(\phi(x) = x)$。 设时间步 $t$ 的输入为单样本 $x_t \in \mathcal{R}^d$，标签为 $y_t$，那么隐藏状态 $h_t\in \mathcal{R}^h$ 的计算表达式为
#### $$ h_t = W_{hx}x_t + W_{hh}h_{t-1}$$
其中，$W_{hx}\in \mathcal{R}^{h\times d}$ 和 $W_{hh} \in \mathcal{R}^{h\times h}$ 是隐藏层权重参数。设输出层权重参数 $W_{qh} \in \mathcal{R}^{q\times h}$，时间步 $t$ 的输出层变量 $O_t\in \mathcal{R}^q$ 计算为
#### $$ O_t = W_{qh}h_t $$
设时间步 $t$ 的损失为 $l(o_t,y_t)$，时间步数为 $T$ 的损失函数 $L$ 定义为
#### $$ L = \frac{1}{T}\sum_{t=1}^T l(O_t,y_t)$$ 
我们将 $L$ 称为有关给定时间步的数据样本的目标函数。

### <font color='blue'>**2、模型计算图**</font>


如图6.3所示，时间步3的隐藏状态 $h_3$的计算依赖模型参数 $W_hx$、$W_hh$、上一时间步隐藏状态 $h_2$ 以及当前时间步输入 $x_3$

<img src = 'CNN3.png' width = 800></img>

### <font color='blue'>**3、方法**</font>

图6.3中的模型参数是 $W_{hx}$， $W_{hh}$和 $W_{qh}$。

与“正向传播” 一节中的类似，训练模型通常需要模型参数的梯度 <font size=5> $\frac{\partial{L}}{\partial{W_{hx}}}$ </font>、<font size=5> $\frac{\partial{L}}{\partial{W_{hh}}}$ </font> 和 <font size=5> $\frac{\partial{L}}{\partial{W_{qh}}}$ </font>。

根据图6.3中的依赖关系，我们可以按照其中箭头所指的反方向依次计算并存储梯度。

首先，目标函数有关各时间步输出层变量的梯度 <font size=4>  $\frac{\partial{L}}{\partial{O_t}} \in \mathcal{R}^q$ </font> 很容易计算：
#### $$ \frac{\partial{L}}{\partial{O_t}} = \frac{\partial l(O_t,y_t)}{T\bullet \partial{O_t}}$$ 
下面，可以计算目标函数有关模型参数 $W_{qh}$ 的梯度 <font size=4> $\frac{\partial L }{\partial W_{qh}} \in \mathcal{R}^{q\times h}$ </font> 。 根据图6.3， $L$ 通过 $O_1,O_2,\cdots,O_T$ 依赖 $W_{qh}$，依赖链式法则：
#### $$ \frac{\partial L}{\partial W_{qh}} = \sum_{t=1}^T prod\big(\frac{\partial L}{\partial O_t},\frac{\partial O_t}{\partial W_{qh}}\big) = \sum_{t=1}^T \frac{\partial L}{\partial O_t} h_t^T$$

其次，我们注意到隐藏状态之间也存在依赖关系。在图6.3中， $L$ 只通过 $O_T$ 依赖最终时间步 $T$ 的隐藏状态 $h_T$。因此，我们先计算目标函数有关最终时间步隐藏状态的梯度 <font size=4>$\frac{\partial L}{\partial h_T} \in \mathcal{R}^h$</font> 。依据链式法则，我们得到
#### $$ \frac{\partial L}{\partial h_T} = prod \big(\frac{\partial L}{\partial O_T}, \frac{\partial O_T}{\partial h_T}\big) = W_{qh}\frac{\partial L}{\partial O_T}$$ 
接下来，对于时间步 $t<T$，在图6.3中， $L$ 通过 $h_{t+1}$ 和 $O_t$ 依赖h_t。依据链式法则，目标函数有关时间步 $t<T$ 的隐藏状态的梯度 $\partial L / \partial h_t\in \mathcal{R}^h$ 需要按照时间步从大到小依次计算：
#### $$ \frac{\partial L}{\partial h_t} = prod\big(\frac{\partial L}{\partial h_{t+1}}, \frac{\partial h_{t+1}}{\partial h_t} \big) + prod\big( \frac{\partial L}{\partial O_t},\frac{\partial O_t}{\partial h_t}\big) = W_{hh}^T \frac{\partial L}{\partial h_{t+1}} + W_{qh}^T \frac{\partial L}{\partial O_t} $$ 

将上面的递归公式展开，对任意时间步 $1\leq t\leq T$，我们可以得到目标函数有关隐藏状态梯度的通项公式：
#### $$ \frac{\partial L}{\partial h_t} = \sum_{i=t}^T(W_{hh}^T)^{T-i}W_{qh}^T \frac{\partial L}{\partial O_{T+t-i}}$$ 
由上式的指数项可见，当时间步 $T$ 较大或 时间步 $t$ 较小时，目标函数有关隐藏状态的梯度容易出现衰减和爆炸。这也会影响其他包含 $\partial L/\partial h_t$ 项的梯度，例如隐藏层中模型参数的梯度 $\partial L/\partial W_{hx}\in \mathcal{R}^{h\times d}$ 和 $\partial L/\partial W_{hh}\in \mathcal{R}^{h\times h}$。在图6.3中， $L$ 通过 $h_1,\cdots,h_T$依赖这些模型参数。依据链式法则，我们有
#### $$\frac{\partial L}{\partial W_{hx}} = \sum_{t=1}^T prod\big(\frac{\partial L}{\partial h_t},\frac{\partial h_t}{\partial W_{hx}}\big) = \sum_{t=1}^T \frac{\partial L}{\partial h_t}x_t^T $$ 
#### $$ \frac{\partial L}{\partial W_{hh}} = \sum_{t=1}^T prod\big(\frac{\partial L}{\partial h_t},\frac{\partial h_t}{\partial W_{hh}}\big) = \sum_{t=1}^T \frac{\partial L}{\partial h_t}h_{t-1}^T$$ 


每次迭代中，我们在依次计算完以上各个梯度后，会将它们存储起来，从而避免重复计算。例如，由于隐藏状态梯度 $\partial L/\partial h_t$ 被计算和存储，之后的模型参数梯度 $\partial L/\partial W_{hx}$ 和$\partial L/\partial W_{hh}$ 的计算可以直接读取 $\partial L/\partial h_t$ 的值而无须重复计算它们。

此外，反向传播中的梯度计算可能会依赖变量的当前值。它们正是通过正向传播计算出来的。例如，参数梯度 $\partial L/\partial W_{hh}$ 的计算需要依赖隐藏状态在时间步 $t=0,\cdots,T-1$ 的当前值 $h_t$ ($h_0$ 是初始化得到的)。这些值，是通过从输入层到输出层的正向传播计算并存储得到的。
